In [30]:
import tensorflow as tf

# 訓練データの読み込み
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "dog_cat_photos/train",
    image_size=(96, 96),
    label_mode="binary",
    batch_size=32,
    shuffle=True
)

# テストデータの読み込み
test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "dog_cat_photos/test",
    image_size=(96, 96),
    label_mode="binary",
    batch_size=32,
    shuffle=False
)

# 画像の水増しをする関数の定義
def flip_left_right(image, label):   # 左右反転
    image = tf.image.flip_left_right(image)
    return image, label

def flip_up_down(image, label):      # 上下反転
    image = tf.image.flip_up_down(image)
    return image, label

def rot90(image, label):             # 反時計回りに90度回転
    image = tf.image.rot90(image)
    return image, label

def rot180(image, label):            # 反時計回りに180度回転
    image = tf.image.rot90(image, k=2)
    return image, label

def rot270(image, label):            # 反時計回りに270度回転
    image = tf.image.rot90(image, k=3)
    return image, label

# 画像の水増し処理の実行
train_dataset_lr     = train_dataset.map(flip_left_right)
train_dataset_ud     = train_dataset.map(flip_up_down)
train_dataset_rot90  = train_dataset.map(rot90)
train_dataset_rot180 = train_dataset.map(rot180)
train_dataset_rot270 = train_dataset.map(rot270)

# 水増ししたデータを訓練データに追加する
train_dataset = train_dataset.concatenate(train_dataset_lr)
train_dataset = train_dataset.concatenate(train_dataset_ud)
train_dataset = train_dataset.concatenate(train_dataset_rot90)
train_dataset = train_dataset.concatenate(train_dataset_rot180)
train_dataset = train_dataset.concatenate(train_dataset_rot270)

# データをシャッフルする
train_dataset = train_dataset.shuffle(32)

# MoboileNetV2を使ってモデルを作成する
input_layer = tf.keras.layers.Input(shape=(96, 96, 3))
l_layer = tf.keras.applications.mobilenet_v2.preprocess_input(input_layer)

base_model = tf.keras.applications.MobileNetV2(
    input_shape=(96, 96, 3),
    input_tensor=l_layer,
    include_top=False,
    weights='imagenet',
    pooling='avg'
)
base_model.trainable = False  # 転移学習のため、事前学習済みモデルの重みを固定

# 出力層を作成
output_layer = tf.keras.layers.Dense(1, activation='sigmoid')

# モデルを作成
model = tf.keras.models.Sequential([
    base_model,
    output_layer
])

# モデルのコンパイル
model.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

# モデルの学習
model.fit(train_dataset, epochs=20)

# テストデータでの評価
model.evaluate(test_dataset)

Found 300 files belonging to 2 classes.
Found 100 files belonging to 2 classes.
Epoch 1/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 4s 40ms/step - accuracy: 0.8500 - loss: 0.3431
Epoch 2/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 2s 40ms/step - accuracy: 0.9433 - loss: 0.1637
Epoch 3/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 3s 44ms/step - accuracy: 0.9622 - loss: 0.1210
Epoch 4/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 3s 41ms/step - accuracy: 0.9728 - loss: 0.0963
Epoch 5/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 3s 41ms/step - accuracy: 0.9828 - loss: 0.0780
Epoch 6/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 3s 42ms/step - accuracy: 0.9872 - loss: 0.0681
Epoch 7/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 3s 43ms/step - accuracy: 0.9911 - loss: 0.0561
Epoch 8/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 3s 44ms/step - accuracy: 0.9928 - loss: 0.0491
Epoch 9/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 3s 43ms/step - accuracy: 0.9939 - loss: 0.0429
Epoch 10/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 3s 40ms/step - accuracy: 0.9950 - loss: 0.0394
Epoch 11/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 3s 40ms/step - accuracy: 0.996

[0.05693339183926582, 0.9599999785423279]